<a href="https://colab.research.google.com/github/Hritikahere/Next-Word-Prediction-using-LSTM/blob/main/Next_word_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries:

In [8]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Embedding, LSTM, Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam
import pickle 
import numpy as np
import os

Load and Pre-Process the data

In [9]:
file = open("blue_castle.txt", "r", encoding = "utf8" )

#store file in list 
lines = []
for i in file:
     lines.append(i)

#Convert list to string 
data = ""
for i in lines:
    data = '  '. join(lines)

#replace unnecessary stuff with space
data = data.replace('\n', '').replace('\r', '').replace('\ufeff', '').replace('“','').replace('”','')  #new line, carriage return, unicode character --> replace by space

#remove unnecessary spaces
data = data.split()
data = ' '.join(data)
data[:500]


'The Project Gutenberg eBook of The Blue Castle, by Lucy Maud Montgomery This eBook is for the use of anyone anywhere in the United States and most other parts of the world at no cost and with almost no restrictions whatsoever. You may copy it, give it away or re-use it under the terms of the Project Gutenberg License included with this eBook or online at www.gutenberg.org. If you are not located in the United States, you will have to check the laws of the country where you are located before usi'

In [10]:
len(data)

402906

Apply tokenization and some other changes

In [11]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([data])

#saving the tokenizer for predict function
pickle.dump(tokenizer, open('token.pkl','wb'))

sequence_data = tokenizer.texts_to_sequences([data])[0]
sequence_data[:15]

[1, 112, 97, 587, 4, 1, 94, 147, 58, 2383, 2384, 1818, 51, 587, 42]

In [12]:
len(sequence_data)

72052

In [13]:
vocab_size = len(tokenizer.word_index) + 1
print(vocab_size)

8413


In [14]:
sequences = []

for i in range (3, len(sequence_data)):
  words = sequence_data[i-3:i+1]
  sequences.append(words)

print("The Length if sequences are:" , len(sequences))
sequences = np.array(sequences)
sequences[:10]

The Length if sequences are: 72049


array([[   1,  112,   97,  587],
       [ 112,   97,  587,    4],
       [  97,  587,    4,    1],
       [ 587,    4,    1,   94],
       [   4,    1,   94,  147],
       [   1,   94,  147,   58],
       [  94,  147,   58, 2383],
       [ 147,   58, 2383, 2384],
       [  58, 2383, 2384, 1818],
       [2383, 2384, 1818,   51]])

In [15]:
X = []
y = []

for i in sequences:
  X.append(i[0:3])
  y.append(i[3])

X = np.array(X)
y = np.array(y)

In [16]:
print("Data:", X[:10])
print("Response:", y[:10])

Data: [[   1  112   97]
 [ 112   97  587]
 [  97  587    4]
 [ 587    4    1]
 [   4    1   94]
 [   1   94  147]
 [  94  147   58]
 [ 147   58 2383]
 [  58 2383 2384]
 [2383 2384 1818]]
Response: [ 587    4    1   94  147   58 2383 2384 1818   51]


In [17]:
y = to_categorical(y, num_classes=vocab_size)
y[:5]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

Creating the model

In [18]:
model = Sequential()
model.add(Embedding(vocab_size, 10, input_length=3))
model.add(LSTM(1000, return_sequences=True))
model.add(LSTM(1000))
model.add(Dense(1000, activation= "relu"))
model.add(Dense(vocab_size, activation="softmax"))

2024-03-12 12:31:46.270069: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-03-12 12:31:46.309631: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 3, 10)             84130     
                                                                 
 lstm (LSTM)                 (None, 3, 1000)           4044000   
                                                                 
 lstm_1 (LSTM)               (None, 1000)              8004000   
                                                                 
 dense (Dense)               (None, 1000)              1001000   
                                                                 
 dense_1 (Dense)             (None, 8413)              8421413   
                                                                 
Total params: 21554543 (82.22 MB)
Trainable params: 21554543 (82.22 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Plot the model

In [20]:
from tensorflow import keras
from keras.utils.vis_utils import plot_model

keras.utils.plot_model(model, to_file='plot.png', show_layer_names=True)

ModuleNotFoundError: No module named 'keras.utils.vis_utils'

Train the model

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

checkpoint = ModelCheckpoint("next_word.h5", monitor='loss', verbise=1, save_best_only=True)
model.compile(loss="categorical_crossentropy", optimizer=Adam(learning_rate=0.001))
model.fit(X, y, epochs=20, batch_size=64, callbacks=[checkpoint])

Epoch 1/20
1126/1126 [==============================] - 33s 21ms/step - loss: 6.7736
Epoch 2/20
1126/1126 [==============================] - 18s 16ms/step - loss: 6.2607
Epoch 3/20
1126/1126 [==============================] - 18s 16ms/step - loss: 5.8552
Epoch 4/20
1126/1126 [==============================] - 19s 17ms/step - loss: 5.5418
Epoch 5/20
1126/1126 [==============================] - 18s 16ms/step - loss: 5.2844
Epoch 6/20
1126/1126 [==============================] - 18s 16ms/step - loss: 5.0524
Epoch 7/20
1126/1126 [==============================] - 18s 16ms/step - loss: 4.8328
Epoch 8/20
1126/1126 [==============================] - 18s 16ms/step - loss: 4.6022
Epoch 9/20
1126/1126 [==============================] - 18s 16ms/step - loss: 4.3600
Epoch 10/20
1126/1126 [==============================] - 18s 16ms/step - loss: 4.1134
Epoch 11/20
1126/1126 [==============================] - 18s 16ms/step - loss: 3.8681
Epoch 12/20
1126/1126 [==============================] - 18s 16

Lets Predict

In [ ]:
from tensorflow.keras.models import load_model
import numpy as np
import pickle

#Load the model and tokenizer
model = load_model('next_word.h5')
tokenizer = pickle.load(open('token.pkl', 'rb'))

def Predict_Next_Words(model, tokenizer, text):

  sequence = tokenizer.texts_to_sequences([text])
  sequence = np.array(sequence)
  preds = np.argmax(model.predict(sequence))
  predict_word = ""

  for key, value in tokenizer.word_index.items():
    if value == preds:
      predicted_word = key
      break

  print(predicted_word)
  return predicted_word

In [ ]:
while(True):
  text = input("Enter your line:")

  if text == "0":
    print("Execution completed....")
    break
  
  else:
    try:
      text: list[str] = text.split(" ")
      text = text[-3:]
      print(text)

      Predict_Next_Words(model, tokenizer, text)

    
    except Exception as e:
       print("Error occured: ",e)
       continue

Enter your line:The Project Gutenberg
['The', 'Project', 'Gutenberg']
1/1 [==============================] - 1s 1s/step
tm
Enter your line:The Project Gutenberg
['The', 'Project', 'Gutenberg']
1/1 [==============================] - 0s 20ms/step
tm
Enter your line:The Project Gutenberg
['The', 'Project', 'Gutenberg']
1/1 [==============================] - 0s 22ms/step
tm
Enter your line:The Project Gutenberg eBook of
['Gutenberg', 'eBook', 'of']
1/1 [==============================] - 0s 21ms/step
the
Enter your line:He was quite
['He', 'was', 'quite']
1/1 [==============================] - 0s 22ms/step
a
Enter your line:however, it may all come to
['all', 'come', 'to']
1/1 [==============================] - 0s 21ms/step
the
